In [172]:
import socket
import time
from multiprocessing import Process, Value
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [173]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 1;
}

//do not read here
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [ ]:
#BUZZ FOR .5 SECONDS
def buzz_active():
    write_gpio(3, 0) #pull low for ON
    time.sleep(.5)
    write_gpio(3, 1) #pull high for OFF
    time.sleep(.5)

def buzz_passive():
    freq = 200
    for i in range(50):
        write_gpio(3, 0)
        time.sleep(0.5/freq)
        write_gpio(3, 1)
        time.sleep(0.5/freq)
    
#-------------------------------------------------   
# SERVER PROCESS
def run_ser

In [185]:
ver(stop_flag):
    #'0.0.0.0' listens on new 192.168.0.120 IP automatically
    for i in range(2): base.leds[i].on()
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.bind(('0.0.0.0', 12345))
        except:
            sock.close()
            sock.bind(('0.0.0.0', 12345))
        sock.listen(10)
        print ("listening...")

        while not stop_flag.value:
            try:
                sock.settimeout(1)
                conn, addr = sock.accept()
                with conn:
                    print(f"Server: connected w/ {addr}")
                    for i in range(4): base.leds[i].on()
                    while not stop_flag.value:
                        data = conn.recv(1024)
                        if not data:
                            break
                        if data.decode() == "BUZZ":
                            print("Server: Buzzing!")
                            buzz_passive()
                        elif data.decode() == "SHUTDOWN":
                            stop_flag.value = True;
            except socket.timeout:
                continue
        sock.close()
        for i in range(4): base.leds[i].off()
                
    
#------------------------------------------------- 
#CLIENT PROCESS
def run_client(target_ip, stop_flag):
    sock = None
    print("Client running. Button 0 to connect.")
    
    while not stop_flag.value:
        # BTN0 = connect
        if btns[0].read() == 1:
            try:
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((target_ip, 12345))
                print("client connected")
                time.sleep(0.25) # debounce
            except:
                print("client: connection failed.")
                if sock:
                    sock.close()
                time.sleep(0.25)

        # BTN1 = send buzz
        if btns[1].read() == 1:
            if sock:
                try:
                    sock.send("BUZZ".encode())
                    print("Buzz!")
                except:
                    print("client: buzz failed, no server connection")
                time.sleep(0.05)
            else:
                print("client: buzz failed, no server connection")

        # BTN2 = disconnect and stop
        if btns[2].read() == 1:
            print("shutting down...")
            if sock:
                try:
                    sock.send("SHUTDOWN".encode())
                    sock.close()
                except:
                    pass
            stop_flag.value = True # end loop

In [ ]:
if __name__ == "__main__":
    write_gpio(3, 1) #start low
    
    OTHER_IP = '192.168.0.192' #IP INPUT HERE
    stop_signal = Value('b', False) # shared flag to kill processes
    
    p1 = Process(target=run_server, args=(stop_signal,))
    p2 = Process(target=run_client, args=(OTHER_IP, stop_signal))
    
    p1.start()
    p2.start()
    
    p1.join()
    print("SERVER SHUTDOWN")
    p2.join()
    print("CLIENT SHUTDOWN")
    
    write_gpio(3, 1)
    print("DONE")

listening...
Client running. Button 0 to connect.
